The purpose of this notebook is to compute the FIM of an NN potential for C with respect to the  weights and biases corresponding to the mapping from the last hidden layer to the output layer.
The FIM is calculated using the posterior to regularize the singularity of the FIM calculated using the likelihood only.
Since we add a prior, then this method is more Bayesian.

In [ ]:
from pathlib import Path
import json
import sys
from tqdm import tqdm
import subprocess
from multiprocessing import Pool


import numpy as np
import scipy
import matplotlib.pyplot as plt
import numdifftools as nd
import torch

from kliff import nn
from kliff.calculators import CalculatorTorch
from kliff.dataset import Dataset
from kliff.dataset.weight import Weight
from kliff.descriptors import SymmetryFunction
from kliff.loss import Loss
from kliff.models import NeuralNetwork

# Random seed
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)
# torch.set_default_tensor_type(torch.DoubleTensor)

%matplotlib inline
plt.style.use("default")

# Setup

## Variables

In [ ]:
# Read setting file
WORK_DIR = Path().absolute()
ROOT_DIR = WORK_DIR.parent
DATA_DIR = ROOT_DIR / "data"
with open(ROOT_DIR / "settings.json", "r") as f:
    settings = json.load(f)
partition = settings["partition"]
suffix = "_".join([str(n) for n in settings["Nnodes"]])
PART_DIR = DATA_DIR / f"{partition}_partition_data"
FP_DIR = PART_DIR / "fingerprints"
RES_DIR = WORK_DIR / "results" / f"{partition}_partition_{suffix}"
if not RES_DIR.exists():
    RES_DIR.mkdir(parents=True)

## Model

In [ ]:
# Architecture
Nlayers = settings["Nlayers"]  # Number of layers, excluding input layer, including outpt layer
Nnodes = settings["Nnodes"]  # Number of nodes per hidden layer
dropout_ratio = 0.0  # Don't use dropout

# Optimizer settings
learning_rate = 1e-3
batch_size = 100
nepochs_total = 40_000  # How many epochs to run in total
nepochs_initial = 2000  # Run this many epochs first
nepochs_save_period = 10  # Then run and save every this many epochs
epoch_change_lr = 5000  # This is the epoch when we change the learning rate

In [ ]:
# Descriptor
descriptor = SymmetryFunction(
    cut_name="cos", cut_dists={"C-C": 5.0}, hyperparams="set51", normalize=True
)
model = NeuralNetwork(descriptor)

# Layers
hidden_layer_mappings = []
for ii in range(Nlayers - 2):
    hidden_layer_mappings.append(nn.Dropout(dropout_ratio))
    hidden_layer_mappings.append(nn.Linear(Nnodes[ii], Nnodes[ii + 1]))
    hidden_layer_mappings.append(nn.Tanh())

model.add_layers(
    # input layer
    nn.Linear(descriptor.get_size(), Nnodes[0]),  # Mapping from input layer to the first
    nn.Tanh(),  # hidden layer
    # hidden layer(s)
    *hidden_layer_mappings,  # Mappings between hidden layers in the middle
    # hidden layer(s)
    nn.Dropout(dropout_ratio),  # Mapping from the last hidden layer to the output layer
    nn.Linear(Nnodes[-1], 1),
    # output layer
)

# Load best model
orig_model_path = (
    ROOT_DIR
    / "training_dropout"
    / "results"
    / "training"
    / f"{partition}_partition_{suffix}"
    / "model_best_train.pkl"
)
model.load(orig_model_path)

## Training set and calculator

In [ ]:
# training set
dataset_path = PART_DIR / "carbon_training_set"
weight = Weight(energy_weight=1.0, forces_weight=np.sqrt(0.1))
tset = Dataset(dataset_path, weight)
configs = tset.get_configs()
nconfigs = len(configs)

# calculator
gpu = False
calc = CalculatorTorch(model, gpu=gpu)
_ = calc.create(
    configs,
    nprocs=20,
    reuse=True,
    fingerprints_filename=FP_DIR / f"fingerprints_train.pkl",
    fingerprints_mean_stdev_filename=FP_DIR / f"fingerprints_train_mean_and_stdev.pkl",
)
bestfit_params = calc.get_opt_params()

In [ ]:
# Loss function
residual_data = {"normalize_by_natoms": True}
loss = Loss(calc, residual_data=residual_data)

# Real calculation

In [ ]:
# loader = calc.get_compute_arguments(batch_size)
loader = calc.get_compute_arguments(1)

## Get reference data

In [ ]:
reference_energy = []
energy_natoms = []
for batch in loader:
    batch_energy = np.array([float(sample["energy"]) for sample in batch])
    batch_natoms = np.array([sample["forces"].shape[0] for sample in batch])
    reference_energy = np.append(reference_energy, batch_energy)
    energy_natoms = np.append(energy_natoms, batch_natoms)
print(len(reference_energy))

In [ ]:
reference_forces = []
forces_natoms = []
for batch in loader:
    batch_forces = [sample["forces"].numpy().flatten() for sample in batch]
    batch_natoms = [
        np.repeat(sample["forces"].shape[0], np.prod(sample["forces"].shape))
        for sample in batch
    ]
    reference_forces = np.append(reference_forces, np.concatenate(batch_forces))
    forces_natoms = np.append(forces_natoms, np.concatenate(batch_natoms))
print(reference_forces.shape)

In [ ]:
# Total number of predictions
npreds = len(reference_energy) + len(reference_forces)

## Compute the Jacobian

Given that we are only considering the parameters of the last (output) layer, then the model is actually linear with respect to these parameters.
The energy of atom $i$ is calculated by
\begin{equation}
    E_i = W \phi_i + b,
\end{equation}
where $W$ is a vector of the weights of the last layer and $b$ is the bias, and $\phi$ is the input to the last layer.
From this, we can see that
\begin{equation}
    \frac{\partial E_i}{\partial w_j} = (\phi_i)_j
    \quad \text{and} \quad
    \frac{\partial E_i}{\partial b} = 1.
\end{equation}
However, note that to compute the configuration energy, we need to sum the energy over all atoms in the configuration.

From this model, we compute the force element of atom $i$ by
\begin{equation}
    F^i_x = \frac{\partial E_i}{\partial r^i_x}
    = \frac{\partial E_i}{\partial \zeta} \frac{\partial \zeta}{\partial r^i_x},
\end{equation}
where $\zeta$ is the atomic descriptor.
Note that using the property of derivative,
\begin{equation}
    \frac{\partial F^i_x}{\partial w_j} = \frac{\partial \zeta}{\partial r^i_x} \frac{\partial}{\partial \zeta} \left( \frac{\partial E_i}{\partial w_j} \right)
    = \frac{\partial \zeta}{\partial r^i_x} \frac{\partial (\phi_i)_j}{\partial \zeta},
\end{equation}
which is the input to the last layer when we use the model to compute the force.
Additionally, notice that
\begin{equation}
    \frac{\partial F^i_x}{\partial b} = 0.
\end{equation}

The FIM for this model is calculated by
\begin{equation}
    \mathcal{I} = J^T J,
\end{equation}
where $J$ is the Jacobian matrix of the _residual_,
\begin{equation}
    r(\theta) = \frac{y - f(\theta)}{\sigma}.
\end{equation}
In this equation $y$ represents the reference data, $f(\theta)$ represents the model predictions, and $\sigma$ is the inverse weight of the residual.
Notice that the element of the Jacobian matrix is
\begin{equation}
    J_{ij} = \frac{1}{\sigma} \frac{\partial f_i}{\partial \theta_j},
\end{equation}
i.e., we need to include th weights in the derivative.

In [ ]:
device = model.device
layers_no_output = model.layers[:-1]

In [ ]:
# Compute the derivative of the energy
de_file = Path(RES_DIR / "jacobian_energy.npy")

if de_file.exists():
    de_all = np.load(de_file)
else:
    energy_weight = 1.0
    # Note that there is still an extra weight factor related to the number of atoms
    de_all = np.empty((0, 129))  # There are 128 weights and 1 bias

    for batch in tqdm(loader):
        # Get the fingerprints - these lines are from calc.compute
        zeta_config = [sample["zeta"] for sample in batch]  # Zeta for each config
        zeta_stacked = torch.cat(zeta_config, dim=0).to(device)
        zeta_stacked.requires_grad_(True)
        natoms_config = [len(zeta) for zeta in zeta_config]  # Number of atoms per config

        # Try to retrieve input values of the last hidden layer
        x_atom = torch.clone(zeta_stacked)
        for layer in layers_no_output:
            x_atom = layer(x_atom)
        x_config = [e.sum(0) for e in torch.split(x_atom, natoms_config)]
        # Derivative of the energy with respect to the weights is just this input values
        de = np.array([elem.detach().numpy() for elem in x_config])
        # Normalize by number of atoms
        de/= np.array(natoms_config).reshape((-1, 1))
        # Add the derivative of energy with respect to the bias
        # Before normalizing by the number of atoms, this value should be the same as
        # the number of atoms. But, we will divide it with the number of atoms anyway.
        de = np.column_stack((de, np.ones(len(de))))
        # Finally, multiply the derivative by the weight
        de *= energy_weight

        # Append to the container matrix
        de_all = np.row_stack((de_all, de))

    # Export
    np.save(RES_DIR / "jacobian_energy.npy", de_all)

In [ ]:
# Compute the derivative of the energy
df_file = Path(RES_DIR / "jacobian_forces.npy")

# Unlike the calculation for the derivative of the energy, this calculation took me
# about 5 minutes. Just loading the data even took about 5s.
if df_file.exists():
    df_all = np.load(df_file)
else:
    forces_weight = np.sqrt(0.1)
    # Note that there is still an extra weight factor related to the number of atoms
    df_all = np.empty((0, 129))  # There are 128 weights and 1 bias

    for batch in tqdm(loader):
        # Get the fingerprints - these lines are from calc.compute
        zeta_config = [sample["zeta"] for sample in batch]  # Zeta for each config
        zeta_stacked = torch.cat(zeta_config, dim=0).to(device)
        zeta_stacked.requires_grad_(True)
        natoms_config = [
            len(zeta) for zeta in zeta_config
        ]  # Number of atoms per config

        # Try to retrieve values from the last hidden layer
        x_atom = torch.clone(zeta_stacked)
        for layer in layers_no_output:
            x_atom = layer(x_atom)

        x_config = [e.sum(0) for e in torch.split(x_atom, natoms_config)]
        # Derivative of the energy with respect to the weights
        de = np.array([elem.detach().numpy() for elem in x_config])
        # Add the derivative of energy with respect to the bias
        de = np.column_stack((de, np.ones(len(de))))

        # Derivative of the forces with respect to all weights of the last hidden layer.
        # This is the same as the input values of the last hidden layer for forces
        # predictions.
        df = []
        for x_atom_node in x_atom.T:
            # de/dzeta for the each node of the last hidden layer
            x_atom_node = x_atom_node.reshape((-1, 1))
            dedzeta = torch.autograd.grad(
                x_atom_node.sum(), zeta_stacked, create_graph=True
            )[0]
            dedzeta_config = torch.split(dedzeta, natoms_config)

            # df/dzeta for the each node of the last hidden layer
            forces_config = []
            for i, sample in enumerate(batch):
                dedzeta = dedzeta_config[i]

                dzetadr_forces = sample["dzetadr_forces"].to(device)
                f = calc._compute_forces(dedzeta, dzetadr_forces)
                # Normalize by number of atoms
                f /= sample["configuration"].get_num_atoms()
                forces_config.append(f)
            forces_config = torch.cat(forces_config)
            df.append(forces_config)

        # Stack the forces
        df = torch.stack(df).T
        zeta_stacked.requires_grad_(False)

        # Convert to numpy
        df = df.detach().numpy()
        # Add the derivative with respect to the bias, which is just zero
        df = np.column_stack((df, np.zeros(len(df))))
        # Finally, multiply the derivative by the weight
        df *= forces_weight

        # Append to the container matrix
        df_all = np.row_stack((df_all, df))

    # Export
    np.save(df_file, df_all)

## Compute the FIM

Derivation of the FIM with using the posterior
* Posterior
    \begin{align*}
        P(\theta | y) &= \frac{L(\theta | y) \pi(\theta)}{P(y)} \\
        L(\theta | y) &= \frac{1}{N_L} \exp{\left[ -\frac{1}{2} \sum_m \left( \frac{y_m - f_m(\theta)}{\sigma_m} \right)^2 \right]} \\
        \pi(\theta) &= \frac{1}{N_\pi} \exp{\left[ -\frac{1}{2} \sum_n \left( \frac{\theta_n - \theta_n^*}{s_n} \right)^2 \right]}
    \end{align*}

* Log-posterior
    \begin{equation*}
        \log [P(\theta | y)] = \log [L(\theta | y)] + \log [\pi(\theta)] - \log [P(y)]
    \end{equation*}
    
* Hessian of log-posterior
    \begin{equation*}
        \frac{\partial^2 \log [P(\theta | y)]}{\partial \theta_\mu \partial \theta_\nu} = \frac{\partial^2 \log [L(\theta | y)]}{\partial \theta_\mu \partial \theta_\nu} + \frac{\partial^2 \log [\pi(\theta)]}{\partial \theta_\mu \partial \theta_\nu}
    \end{equation*}
    $P(y)$ is independent of $\theta$.
    
    * Hessian of log-likelihood
        \begin{align*}
            \log [L(\theta | y)] &= -\log(N_L) -\frac{1}{2} \sum_m \left( \frac{y_m - f_m(\theta)}{\sigma_m} \right)^2 \\
            \frac{\partial \log [L(\theta | y)]}{\partial \theta_\mu} &= \sum_m \left( \frac{y_m - f_m(\theta)}{\sigma_m^2} \right) \left( \frac{\partial f_m(\theta)}{\partial \theta_\mu} \right) \\
            \frac{\partial^2 \log [L(\theta | y)]}{\partial \theta_\mu \partial \theta_\nu} &= \sum_m \left[ -\frac{1}{\sigma_m^2} \frac{\partial f_m(\theta)}{\partial \theta_\mu} \frac{\partial f_m(\theta)}{\partial \theta_\nu} + \left( \frac{y_m - f_m(\theta)}{\sigma_m^2} \right) \frac{\partial^2 f_m(\theta)}{\partial \theta_\mu \partial \theta_\nu} \right]
        \end{align*}

    * Hessian of log-prior
        \begin{align*}
            \log [\pi(\theta)] &= \log(N_\pi) -\frac{1}{2} \sum_n \left( \frac{\theta_n - \theta_n^*}{s_n} \right)^2 \\
            \frac{\partial \log [\pi(\theta)]}{\partial \theta_\mu} &= -\left( \frac{\theta_\mu - \theta_\mu^*}{s_\mu^2} \right) \\
            \frac{\partial^2 \log [\pi(\theta)]}{\partial \theta_\mu \partial \theta_\nu} &= - \frac{\delta_{\mu \nu}}{s_\mu^2}
        \end{align*}
        
* Negative expectation value of the Hessian - FIM
    \begin{align*}
        -\left< \frac{\partial^2 \log [P(\theta | y)]}{\partial \theta_\mu \partial \theta_\nu} \right> &= -\left< \frac{\partial^2 \log [L(\theta | y)]}{\partial \theta_\mu \partial \theta_\nu} \right> - \left< \frac{\partial^2 \log [\pi(\theta)]}{\partial \theta_\mu \partial \theta_\nu} \right> \\
        &= \sum_m \left[ \frac{1}{\sigma_m^2} \frac{\partial f_m(\theta)}{\partial \theta_\mu} \frac{\partial f_m(\theta)}{\partial \theta_\nu} \right] + \frac{\delta_{\mu \nu}}{s_\mu^2}
    \end{align*}

In [ ]:
# Log-likelihood term
fim_energy = de_all.T @ de_all
fim_forces = df_all.T @ df_all
fim_like = fim_energy + fim_forces

In [ ]:
# Log-prior term - A Xavier normal prior, which is the distribution we use to sample
# the initial weights and biases. However, we will center it around the best fit,
# instead of the origin.
fan_in = model.layers[-1].in_features  # Number of input of the last layer
fan_out = model.layers[-1].out_features  # Number of output of the last layer
s = np.sqrt(2 / (fan_in + fan_out))  # standard deviation of the Xavier normal prior
fim_prior = np.diag(np.ones(len(fim_like)) / (s ** 2))

# # Log-prior term - Standard normal distribution
# fim_prior = np.diag(np.ones(len(fim_like)))

In [ ]:
# FIM using posterior
# Temperature term
nparams = 129
C0 = 0.5 * loss._get_loss_epoch(loader)  # This loss value doesn't have a factor of 1/2
# T = 2 * C0 / nparams  # Natural temperature
# T = 2 * C0 / (npreds - nparams)  # Unbiased estimate of variance
T = 1.0
print("Use temperature hyperparameter", T)
# FIM using posterior
fim_configs = fim_like / T + fim_prior

In [ ]:
# Compare the eigenvalues
eigvals_like = scipy.linalg.eigvalsh(fim_like)
eigvals_post = scipy.linalg.eigvalsh(fim_configs)
# print(eigvals)

plt.figure()
for lam1, lam2 in zip(eigvals_like, eigvals_post):
    plt.plot([-0.4, 0.4], [lam1, lam1], c="k")
    plt.plot([0.6, 1.4], [lam2, lam2], c="b")
plt.yscale("log")
plt.xticks([0, 1], ["Likelihood", "Posterior"])
plt.ylabel("Eigenvalue")
plt.show()

# Model Ensemble

In [ ]:
# Covariance
cov = scipy.linalg.pinvh(fim_configs)
np.save(RES_DIR / "covariance_parameters.npy", cov)
# scipy.linalg.eigvalsh(cov)

# plt.figure()
# plt.plot(np.diag(cov))
# plt.yscale("log")
# plt.ylim(1e-6, 1)
# plt.show()

In [ ]:
# Parameter samples
p0 = bestfit_params[-nparams:]

param_samples_file = RES_DIR / "samples_bayes_parameters.npy"
if param_samples_file.exists():
    param_samples = np.load(param_samples_file)
else:
    param_samples = np.random.multivariate_normal(p0, cov, 100)
    np.save(param_samples_file, param_samples)

In [ ]:
def install_uninstall_model(modelname, mode="install"):
    """This function will install or remove KIM model."""
    kim_command = "kim-api-collections-management"
    if mode == "install":
        flags = ["install", "user"]
    elif mode == "uninstall":
        flags = ["remove", "--force"]
        modelname = Path(modelname).name
    command = np.concatenate(([kim_command], flags, [modelname]))
    subprocess.run(command)

In [ ]:
# Write and install model ensembles
for ii, params in tqdm(enumerate(param_samples), total=len(param_samples)):
    # Write the model
    complete_params = bestfit_params.copy()
    complete_params[-nparams:] = params
    calc.update_model_params(complete_params)  # Update parameters
    # Install model
    modelpath = RES_DIR / f"{ii:03d}" / f"DUNN_C_fimbayes_{ii:03d}"
    model.write_kim_model(modelpath)  # Write

In [ ]:
def install_uninstall_wrapper(ii):
    modelname = RES_DIR / f"{ii:03d}" / f"DUNN_C_fimbayes_{ii:03d}"
    # Uninstall first to make sure we don't use the wrong models later
    install_uninstall_model(modelname, "uninstall")
    # Install this model ensemble
    install_uninstall_model(modelname, "install")


with Pool(25) as p:
    p.map(install_uninstall_wrapper, range(100))